In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Electra 모델 클래스
class ElectraVectorizer:
    def __init__(self, model_name):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name).to(self.device)

    def get_first_hidden_state(self, text):
        inputs = self.tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(self.device)
        #tokens = vectorizer.tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
        #print("Tokens:", tokens)
        #print("Token IDs:", inputs['input_ids'][0])
        with torch.no_grad():
            outputs = self.model(**inputs)
            #print(outputs.last_hidden_state[0].size())
        return outputs.last_hidden_state[0][0].cpu().numpy()  # 첫 번째 hidden state 벡터를 사용

# 데이터 로드
books_df = pd.read_excel('/content/songs_with_emotions.xlsx')  # 가사 데이터 로드
book_texts = books_df['lyrics'].tolist()

# 모델 초기화
model_name = "blue2959/koelectra-base-v3-discriminator-finetuned-kor-8-emotions_v1.4"
vectorizer = ElectraVectorizer(model_name)

# 도서 벡터 미리 계산하여 저장
book_vectors = [vectorizer.get_first_hidden_state(text) for text in book_texts]


In [ ]:
# 일기 벡터 계산
diary_text = "오늘은 아침부터 몸을 풀기 위해 자전거를 타고 동네수영장을 다녀왔어. 수영을 하면서 사람들이 많아서 조금 불편했지만, 오랜만에 수영을 하니까 정말 좋았어. 수영을 마치고 컵라면을 먹었는데, 정말 맛있었어. 이후에는 '나는 솔로'를 봤는데, 별로 재미있지는 않았어. 그래도 오늘 하루는 전반적으로 만족스러웠어."
diary_vector = vectorizer.get_first_hidden_state(diary_text)
# 코사인 유사도 계산 및 상위 3개의 도서 선택
similarities = []
idx = 0
for book_vector in book_vectors:
    similarity = cosine_similarity(diary_vector.reshape(1, -1), book_vector.reshape(1, -1))
    similarities.append((similarity[0][0], idx))
    idx += 1

# 유사도가 높은 상위 3개의 음악 선택
top_indices = sorted(similarities, key=lambda x : (x[0], x[1]))
top_indices = [top_indices[-i][1] for i in range(1, 6)]
top_books = books_df.iloc[top_indices]

for index in top_indices:
  print(similarities[index])

# 결과 출력
print("유사도가 높은 상위 3개의 음악:")
print(top_books[['song_title','artist','genre', 'lyrics','emotion']])

(0.8952471, 11666)
(0.8899802, 11683)
(0.8754647, 8292)
(0.87294996, 17414)
(0.8706979, 17118)
유사도가 높은 상위 3개의 음악:
                       song_title                     artist     genre  \
11666  Kona (Feat. Jennifer Choi)                 90yonge st  R&B/Soul   
11683                   Moondance  플랫폼 스테레오(Platform Stereo)      인디음악   
8292                          dot                      Meego  R&B/Soul   
17414                         바다님             양반들 (Yangbans)      인디음악   
17118                         춤의왕                        차세대      인디음악   

                                                                                                                                                                                                                                                                                                                                                                                                                                lyrics  \
11666          

In [ ]:
top_books['lyrics']

11666                                                                                                                                                                                         푸른 빛 물결위로 비춰진 summer memories 칵테일 돌고래 이 여름에 머물거야 후줄근한 셔츠에 시원한 바람 파도소리와 다 갖췄어 너만 빼고 다 갖췄어 너만 빼고 아주 멀리 구름사이로 비추는 summer memories 익숙한듯 낯선바다 다시 찾아올것 같아 후줄근한 셔츠에 시원한 바람 파도소리와 다 갖췄어 너만 빼고 후줄근한 셔츠에 시원한 바람 파도소리와 다 갖췄어 너만 빼고 
11847                                                                                                                                                         물속에 비친 너는 흐리게도 선명하다 손에 물을 가득 담아 너를 한껏 비추어 본다 Moon in water Wave in water Moon in water Wave in water 물속에 비친 너는 여전히 빛이 나고 손에 물을 가득 담아 너를 한껏 비추어 본다 Moon in water 그래 이대로 내 손에 잠겨있어 줘요 Wave in water 하늘 위로 헤엄쳐 갈 수 있으면 해 Moon in water 그래 이대로 항상 내 곁에 있어줘요 
11683                                                                                                                                                                                     

In [ ]:
# 출력 제한 설정 해제
pd.set_option('display.max_colwidth', None)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification


int2emo = {0: "기쁨", 1: "당황",  2: "분노", 3:"불안", 4:"상처", 5: "슬픔", 6: "중립"}
class ElectraSentimentClassifier:
    def __init__(self):
        self.model_name = "blue2959/koelectra-base-v3-discriminator-finetuned-kor-8-emotions_v1.4"
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        self.model = AutoModelForSequenceClassification.from_pretrained(self.model_name).to(self.device)

    def predict_sentiment(self, conversation):
        inputs = self.tokenizer(conversation, return_tensors="pt", padding=True, truncation=True, max_length=512).to(self.device)
        with torch.no_grad():
            outputs = self.model(**inputs)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1).item()
        probabilities = torch.nn.functional.softmax(logits, dim=-1)[0].tolist()
        self.print_result(conversation, probabilities, predictions)

        return predictions, probabilities  # Ensure int2emo[predictions] is returned correctly

    def ready_model(self):
        self.model.to(self.device)

    def unload_model(self):
        self.model.cpu()
        torch.cuda.empty_cache()
        del self.model
        self.model = None

    def print_result(self, sentence, probabilities, predictions):
        # 터미널에 출력
        print("==== Model Prediction ====")
        print(f"Input Sentence: {sentence}")
        print("Probabilities:")
        for i, prob in enumerate(probabilities):
            print(f"  {int2emo[i]}: {prob * 100:.1f}%")
        predicted_label = int2emo[predictions]  # Add this line to define predicted_label
        print(f"Predicted Label: {predicted_label} ({predictions})")
        print("==========================")

In [ ]:
import pandas as pd
pd.options.display.float_format = '{:.6f}'.format
import numpy as np
import random  # 랜덤 모듈 임포트
import os

import logging

# 로그 생성
logger = logging.getLogger()

# 로그의 출력 기준 설정
logger.setLevel(logging.INFO)

# log 출력 형식
formatter = logging.Formatter("%(message)s")

# Start defining and assigning your handlers here
handler = logging.StreamHandler()
handler.setFormatter(formatter)
logger.addHandler(handler)

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
import re

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# 도서 목록 로드
file_path = '/content/kor_song_tag.csv'  # 파일 업로드 후 경로 변경 필요
book_list = pd.read_csv(file_path, sep='\t')

# 감정 분석 클래스 인스턴스 생성
classifier = ElectraSentimentClassifier()

# 도서 소개 텍스트 감정 분석
def analyze_book_emotions(book_list):
    book_emotions = {}
    emotions = []
    for index, row in book_list.iterrows():
        introduction = row['lyrics']
        prediction, _ = classifier.predict_sentiment(introduction)
        book_emotions[row['lyrics']] = int2emo[prediction]
        emotions.append(int2emo[prediction])
    book_list['emotion2'] = emotions
    return book_emotions

# 도서 소개 텍스트 감정 분석 결과
book_emotions = analyze_book_emotions(book_list)

# 감정에 따른 도서 추천 함수
def recommend_books_by_emotion(emotion, book_emotions):
    recommended_books = [book for book, book_emotion in book_emotions.items() if book_emotion == emotion]
    if len(recommended_books) > 5:
        recommended_books = random.sample(recommended_books, 5)
    return recommended_books

# 사용 예제
diary_entry = "오늘 나는 피자와 치킨을 먹었고, 피곤해서 낮잠을 잤다. 나는 어제 늦게 3시에 들어갔기 때문에 피곤해서 낮잠을 자게 되었다. 낮잠을 자고 나니까 확실히 훨씬 나았다. 한 30분 정도 자고, 어제 한 4시간 동안 차를 타다보니 피곤해서 낮잠을 자게 되었다. 오늘은 좋은 낮잠을 했고, 내 기분도 좋아졌다. 오늘 하루도 수고했다."
prediction, _ = classifier.predict_sentiment(diary_entry)
emotion = int2emo[prediction]
recommended_books = recommend_books_by_emotion(emotion, book_emotions)

print(f"감정: {emotion}")
print("추천 도서:")
for book in recommended_books:
    print(book)

# 결과를 엑셀 파일로 저장
output_file_path = '/content/kor_tag2_with_emotions.xlsx'
book_list.to_excel(output_file_path, index=False)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  슬픔: 33.0%
  중립: 0.1%
Predicted Label: 기쁨 (0)
==== Model Prediction ====
Input Sentence: 서쪽하늘로 노을은 지고 이젠 슬픔이 돼버린 그대를 다시 부를수 없을 것 같아 또 한번 불러보네 소리쳐 불러도 늘 허공에 부서져 돌아오는 너의 이름 이젠 더 견딜 힘조차 없게 날 버려두고 가지 사랑하는 날 떠나가는 날 하늘도 슬퍼서 울어준 날 빗속에 떠날 나였음을 넌 알고 있는듯이 비가오는 날엔 난 항상 널 그리워해 언젠간 널 다시 만나는 그 날을 기다리며 비내린 하늘은 왜그리 날 슬프게해 흩어진 내 눈물로 널 잊고 싶은데 비가오는 날엔 난 항상 널 그리워해 언젠간 널 다시 만나는 그 날을 기다리며 비내린 하늘은 왜그리 날 슬프게해 흩어진 내 눈물로 널 잊고 싶은데 가고싶어 널 보고싶어 꼭 찾고 싶었어 하지만 너의 모습은 아직도 그 자리에 비가오는 날엔 난 항상 널 그리워해 언젠간 널 다시 만나는 그 날을 기다리며 비내린 하늘은 왜그리 날 슬프게해 흩어진 내 눈물로 널 잊고 싶은데 하지만 난 널 사랑해 
Probabilities:
  기쁨: 0.1%
  당황: 0.3%
  분노: 1.3%
  불안: 0.7%
  상처: 2.3%
  슬픔: 95.3%
  중립: 0.0%
Predicted Label: 슬픔 (5)
==== Model Prediction ====
Input Sentence: 꼭 그러지 않아도 충분히 널 이해할 수 있어 다른 사람 곁에 서 있는 니 모습이 조금 어색하지만 다 버리지 않아도 어떻게든 이겨낼 수 있어 다른 사랑 찾아가버린 니 얼굴이 그렇게 밉진 않아 이제 우리 같은 시간 속을 남처럼 그렇게 걸으면 돼 달아나도 가지 못할 기억 안고 아무말도 하지마요 더는 안된다는거 잘 알아요 많은 날들이 아무 의미없진 않겠죠 멀어지는 바람처럼 우리 함께한 기억들을 꿈처럼 그렇게 안으면 돼 눈 감아도 잊지 못할 추억의 널 묻고 아무말도 하

In [ ]:
recommended_books[0:3]

['솔직히 말할게 많이 기다려 왔어 너도 그랬을 거라 믿어 오늘이 오길 매일같이 달력을 보면서 솔직히 나에게도 지금 이 순간은 꿈만 같아 너와 함께라 오늘을 위해 꽤 많은 걸 준비해 봤어 All about you and I 다른 건 다 제쳐 두고 Now come with me Take my hand 아름다운 청춘의 한 장 함께 써내려 가자 너와의 추억들로 가득 채울래 Come on 아무 걱정도 하지는 마 나에게 다 맡겨 봐 지금 이 순간이 다시 넘겨볼 수 있는 한 페이지가 될 수 있게 This is our page Our page 솔직히 말할게 지금이 오기까지 마냥 순탄하진 않았지 오늘이 오길 나도 목 빠져라 기다렸어 솔직히 나보다도 네가 몇 배는 더 힘들었을 거라고 믿어 오늘을 위해 그저 견뎌줘서 고마워 All about you and I 다른 건 다 제쳐 두고 Now come with me Take my hand 아름다운 청춘의 한 장 함께 써내려 가자 너와의 추억들로 가득 채울래 Come on 아무 걱정도 하지는 마 나에게 다 맡겨 봐 지금 이 순간이 다시 넘겨볼 수 있는 한 페이지가 될 수 있게 Want you to Come on out and have fun Want us to Have the time of our life Oh 너와의 추억들로 가득 채울래 Come on 아무 걱정도 하지는 마 나에게 다 맡겨 봐 지금 이 순간이 다시 넘겨볼 수 있는 한 페이지가 될 수 있게 This is our page Our page ',
 '이윽고 내가 한눈에 너를 알아봤을 때 모든 건 분명 달라지고 있었어 내 세상은 널 알기 전과 후로 나뉘어 니가 숨 쉬면 따스한 바람이 불어와 니가 웃으면 눈부신 햇살이 비춰 거기 있어줘서 그게 너라서 가끔 내 어깨에 가만히 기대주어서 나는 있잖아 정말 빈틈없이 행복해 너를 따라서 시간은 흐르고 멈춰 물끄러미 너를 들여다 보곤 해 그것 말고는 아무것도 할 수 없어서 너의 모든 순간 그게 나였으면 좋겠다 생각만 해도 가슴이 차올라

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification


int2emo = {0: "기쁨", 1: "당황",  2: "분노", 3:"불안", 4:"상처", 5: "슬픔", 6: "중립"}
class ElectraSentimentClassifier:
    def __init__(self):
        self.model_name = "blue2959/koelectra-base-v3-discriminator-finetuned-kor-8-emotions_v1.4"
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        self.model = AutoModelForSequenceClassification.from_pretrained(self.model_name).to(self.device)

    def predict_sentiment(self, conversation):
        inputs = self.tokenizer(conversation, return_tensors="pt", padding=True, truncation=True, max_length=512).to(self.device)
        with torch.no_grad():
            outputs = self.model(**inputs)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1).item()
        probabilities = torch.nn.functional.softmax(logits, dim=-1)[0].tolist()
        self.print_result(conversation, probabilities, predictions)

        return predictions, probabilities  # Ensure int2emo[predictions] is returned correctly

    def ready_model(self):
        self.model.to(self.device)

    def unload_model(self):
        self.model.cpu()
        torch.cuda.empty_cache()
        del self.model
        self.model = None

    def print_result(self, sentence, probabilities, predictions):
        # 터미널에 출력
        print("==== Model Prediction ====")
        print(f"Input Sentence: {sentence}")
        print("Probabilities:")
        for i, prob in enumerate(probabilities):
            print(f"  {int2emo[i]}: {prob * 100:.1f}%")
        predicted_label = int2emo[predictions]  # Add this line to define predicted_label
        print(f"Predicted Label: {predicted_label} ({predictions})")
        print("==========================")

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Electra 모델 클래스
class ElectraVectorizer:
    def __init__(self, model_name):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name).to(self.device)

    def get_first_hidden_state(self, text):
        inputs = self.tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(self.device)
        #tokens = vectorizer.tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
        #print("Tokens:", tokens)
        #print("Token IDs:", inputs['input_ids'][0])
        with torch.no_grad():
            outputs = self.model(**inputs)
            #print(outputs.last_hidden_state[0].size())
        return outputs.last_hidden_state[0][0].cpu().numpy()  # 첫 번째 hidden state 벡터를 사용


In [ ]:
# 모델 초기화
model_name = "blue2959/koelectra-base-v3-discriminator-finetuned-kor-8-emotions_v1.4"
vectorizer = ElectraVectorizer(model_name)
# 일기 벡터 계산
diary_text = "사랑해서 웃음이 나와 사랑하기 싫어요"
diary_vector = vectorizer.get_first_hidden_state(diary_text)

In [ ]:
# 감정 분석 클래스 인스턴스 생성
classifier = ElectraSentimentClassifier()

# 사용 예제
prediction, _ = classifier.predict_sentiment(diary_text)
emotion = int2emo[prediction]

print(f"감정: {emotion}")

==== Model Prediction ====
Input Sentence: 사랑해서 웃음이 나와 사랑하기 싫어요
Probabilities:
  기쁨: 0.4%
  당황: 0.0%
  분노: 3.1%
  불안: 0.1%
  상처: 0.0%
  슬픔: 93.1%
  중립: 3.1%
Predicted Label: 슬픔 (5)
감정: 슬픔


노래 탑100

In [ ]:
# 데이터 로드
books_df2 = pd.read_excel('/content/song_all_with_emotions.xlsx')  # 가사 데이터 로드
book_texts2 = books_df2['lyric'].tolist()

# 모델 초기화
model_name = "blue2959/koelectra-base-v3-discriminator-finetuned-kor-8-emotions_v1.4"
vectorizer = ElectraVectorizer(model_name)

# 도서 벡터 미리 계산하여 저장
book_vectors2 = [vectorizer.get_first_hidden_state(text) for text in book_texts2]

In [ ]:
# 코사인 유사도 계산 및 상위 3개의 도서 선택
similarities2 = []
idx = 0
for book_vector in book_vectors2:
    similarity2 = cosine_similarity(diary_vector.reshape(1, -1), book_vector.reshape(1, -1))
    similarities2.append((similarity2[0][0], idx))
    idx += 1

# 유사도가 높은 상위 3개의 음악 선택
top_indices2 = sorted(similarities2, key=lambda x : (x[0], x[1]))
top_indices2 = [top_indices2[-i][1] for i in range(1, 6)]
top_books2 = books_df2.iloc[top_indices2]

for index in top_indices2:
  print(similarities2[index])

# 결과 출력
print("유사도가 높은 상위 3개의 음악:")
print(top_books2[['song_name','artist', 'lyric','emotion']])

(0.761199, 630)
(0.75984687, 787)
(0.7544346, 430)
(0.75278854, 745)
(0.7390189, 727)
유사도가 높은 상위 3개의 음악:
             song_name        artist  \
630         화려하지 않은 고백  규현 (KYUHYUN)   
787   늦은 밤 헤어지긴 너무 아쉬워   케이시 (Kassy)   
430                밤편지           아이유   
745  그때 그 순간 그대로 (그그그)  WSG워너비 (가야G)   
727       별빛 같은 나의 사랑아           임영웅   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

전체 제일 많은 곡

In [ ]:
# 데이터 로드
books_df = pd.read_csv('/content/test_songs.csv', sep='\t')  # 가사 데이터 로드
book_texts = books_df['lyrics'].tolist()

# 모델 초기화
model_name = "blue2959/koelectra-base-v3-discriminator-finetuned-kor-8-emotions_v1.4"
vectorizer = ElectraVectorizer(model_name)

# 도서 벡터 미리 계산하여 저장
book_vectors = [vectorizer.get_first_hidden_state(text) for text in book_texts]

In [ ]:
# 코사인 유사도 계산 및 상위 3개의 도서 선택
similarities = []
idx = 0
for book_vector in book_vectors:
    similarity = cosine_similarity(diary_vector.reshape(1, -1), book_vector.reshape(1, -1))
    similarities.append((similarity[0][0], idx))
    idx += 1

# 유사도가 높은 상위 3개의 음악 선택
top_indices = sorted(similarities, key=lambda x : (x[0], x[1]))
top_indices = [top_indices[-i][1] for i in range(1, 6)]
top_books = books_df.iloc[top_indices]

for index in top_indices:
  print(similarities[index])

# 결과 출력
print("유사도가 높은 상위 3개의 음악:")
print(top_books[['song_title','artist','genre', 'lyrics','emotion','tag','plylstseq']])

(0.86998737, 14870)
(0.8506135, 13689)
(0.8328817, 9549)
(0.82224727, 4678)
(0.8213995, 5150)
유사도가 높은 상위 3개의 음악:
                    song_title         artist genre  \
14870              당신의 푸른 눈가에서            2단지  인디음악   
13689         별자리 (Good Night)          NCT U   발라드   
9549                       오롯이    헤이즈 (Heize)   발라드   
4678   ALL MY LOVE Korean Ver.            박보검   발라드   
5150           4season (Outro)  마마무 (Mamamoo)    댄스   

                                                                                                                                                                                                                                                                                                                                                                                                                                                        lyrics  \
14870                                                                                                     

태그가 5개 이상인 것들만

In [ ]:
# 데이터 로드
books_df3 = pd.read_csv('/content/songs_with_tags.csv', sep='\t')  # 가사 데이터 로드
book_texts3 = books_df3['lyrics'].tolist()

# 모델 초기화
model_name = "blue2959/koelectra-base-v3-discriminator-finetuned-kor-8-emotions_v1.4"
vectorizer = ElectraVectorizer(model_name)

# 도서 벡터 미리 계산하여 저장
book_vectors3 = [vectorizer.get_first_hidden_state(text) for text in book_texts3]

In [ ]:
# 코사인 유사도 계산 및 상위 3개의 도서 선택
similarities3 = []
idx = 0
for book_vector in book_vectors3:
    similarity3 = cosine_similarity(diary_vector.reshape(1, -1), book_vector.reshape(1, -1))
    similarities3.append((similarity3[0][0], idx))
    idx += 1

# 유사도가 높은 상위 3개의 음악 선택
top_indices3 = sorted(similarities3, key=lambda x : (x[0], x[1]))
top_indices3 = [top_indices3[-i][1] for i in range(1, 6)]
top_books3 = books_df3.iloc[top_indices3]

for index in top_indices3:
  print(similarities3[index])

# 결과 출력
print("유사도가 높은 상위 3개의 음악:")
print(top_books3[['song_title','artist','genre', 'lyrics','emotion','tag','plylstseq']])

(0.8328817, 3178)
(0.82224727, 2015)
(0.79471123, 2897)
(0.7923041, 1106)
(0.78755754, 3489)
유사도가 높은 상위 3개의 음악:
                   song_title       artist     genre  \
3178                      오롯이  헤이즈 (Heize)       발라드   
2015  ALL MY LOVE Korean Ver.          박보검       발라드   
2897                  우리가 모르는          김사월    포크/블루스   
1106      Welcome to the Show  DAY6 (데이식스)      록/메탈   
3489                  Your/My          박재범  R&B/Soul   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        lyrics  \
3178 

태그가 5개 이상이고 플레이리스트에 10번이상 들어간 곡

In [ ]:
# 데이터 로드
books_df4 = pd.read_csv('/content/songs_with_tags2.csv', sep='\t')  # 가사 데이터 로드
book_texts4 = books_df4['lyrics'].tolist()

# 모델 초기화
model_name = "blue2959/koelectra-base-v3-discriminator-finetuned-kor-8-emotions_v1.4"
vectorizer = ElectraVectorizer(model_name)

# 도서 벡터 미리 계산하여 저장
book_vectors4 = [vectorizer.get_first_hidden_state(text) for text in book_texts4]

In [ ]:
# 코사인 유사도 계산 및 상위 3개의 도서 선택
similarities4 = []
idx = 0
for book_vector in book_vectors4:
    similarity4 = cosine_similarity(diary_vector.reshape(1, -1), book_vector.reshape(1, -1))
    similarities4.append((similarity4[0][0], idx))
    idx += 1

# 유사도가 높은 상위 3개의 음악 선택
top_indices4 = sorted(similarities4, key=lambda x : (x[0], x[1]))
top_indices4 = [top_indices4[-i][1] for i in range(1, 6)]
top_books4 = books_df4.iloc[top_indices4]

for index in top_indices4:
  print(similarities4[index])

# 결과 출력
print("유사도가 높은 상위 3개의 음악:")
print(top_books4[['song_title','artist','genre', 'lyrics','emotion','tag','plylstseq']])

(0.7923041, 357)
(0.7631839, 491)
(0.7378099, 233)
(0.7376559, 494)
(0.7373312, 507)
유사도가 높은 상위 3개의 음악:
              song_title                 artist     genre  \
357  Welcome to the Show            DAY6 (데이식스)      록/메탈   
491                  여름비           샘김 (Sam Kim)    포크/블루스   
233              언제나 사랑해            케이시 (Kassy)       발라드   
494               josee!  데이먼스 이어 (Damons year)  R&B/Soul   
507                   위로                    권진아       발라드   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                 lyrics  \
357  이젠 혼자가 아닐 무대 너무나 감격스러워 끝없는 가능성 중에 날 골라줘서 고마워 나와 맞이하는 미래가 위태로울지도 몰

태그가 5개 이상이고 플레이리스트에 10번이상 들어간 곡을 태그로 분석



In [ ]:
# 데이터 로드
books_df5 = pd.read_excel('/content/tag2_with_emotions.xlsx')  # 가사 데이터 로드
book_texts5 = books_df5['tag'].tolist()

# 모델 초기화
model_name = "blue2959/koelectra-base-v3-discriminator-finetuned-kor-8-emotions_v1.4"
vectorizer = ElectraVectorizer(model_name)

# 도서 벡터 미리 계산하여 저장
book_vectors5 = [vectorizer.get_first_hidden_state(text) for text in book_texts5]

In [ ]:
# 코사인 유사도 계산 및 상위 3개의 도서 선택
similarities5 = []
idx = 0
for book_vector in book_vectors5:
    similarity5 = cosine_similarity(diary_vector.reshape(1, -1), book_vector.reshape(1, -1))
    similarities5.append((similarity5[0][0], idx))
    idx += 1

# 유사도가 높은 상위 3개의 음악 선택
top_indices5 = sorted(similarities5, key=lambda x : (x[0], x[1]))
top_indices5 = [top_indices5[-i][1] for i in range(1, 6)]
top_books5 = books_df5.iloc[top_indices5]

for index in top_indices5:
  print(similarities5[index])

# 결과 출력
print("유사도가 높은 상위 3개의 음악:")
print(top_books5[['song_title','artist','genre', 'lyrics','emotion2','tag','plylstseq']])

(0.7085501, 187)
(0.70687336, 167)
(0.6912029, 500)
(0.68882143, 513)
(0.68454945, 592)
유사도가 높은 상위 3개의 음악:
                       song_title  artist     genre  \
187                           말해줘     지누션        댄스   
167                     Celebrity     아이유        댄스   
500             잊어버리지마 (Feat. 태연)   Crush  R&B/Soul   
513  나비와 고양이 (feat.백현 (BAEKHYUN))  볼빨간사춘기      인디음악   
592                             와     이정현        댄스   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

한국가사만 추출한 태그가 5개 이상이고 플레이리스트에 10번이상 들어간 곡

In [ ]:
# 데이터 로드
books_df6 = pd.read_excel('/content/kor_tag2_with_emotions.xlsx')  # 가사 데이터 로드
book_texts6 = books_df6['lyrics'].tolist()

# 모델 초기화
model_name = "blue2959/koelectra-base-v3-discriminator-finetuned-kor-8-emotions_v1.4"
vectorizer = ElectraVectorizer(model_name)

# 도서 벡터 미리 계산하여 저장
book_vectors6 = [vectorizer.get_first_hidden_state(text) for text in book_texts6]

In [ ]:
# 코사인 유사도 계산 및 상위 3개의 도서 선택
similarities6 = []
idx = 0
for book_vector in book_vectors6:
    similarity6 = cosine_similarity(diary_vector.reshape(1, -1), book_vector.reshape(1, -1))
    similarities6.append((similarity6[0][0], idx))
    idx += 1

# 유사도가 높은 상위 3개의 음악 선택
top_indices6 = sorted(similarities6, key=lambda x : (x[0], x[1]))
top_indices6 = [top_indices6[-i][1] for i in range(1, 6)]
top_books6 = books_df6.iloc[top_indices6]

for index in top_indices6:
  print(similarities6[index])

# 결과 출력
print("유사도가 높은 상위 3개의 음악:")
print(top_books6[['song_title','artist','genre', 'lyrics','emotion2','tag','plylstseq']])

(0.79922515, 357)
(0.7789341, 388)
(0.758354, 491)
(0.7482853, 397)
(0.7378099, 233)
유사도가 높은 상위 3개의 음악:
              song_title        artist   genre  \
357  Welcome to the Show   DAY6 (데이식스)    록/메탈   
388       Boom Boom Bass         RIIZE      댄스   
491                  여름비  샘김 (Sam Kim)  포크/블루스   
397        Shooting Star  Kep1er (케플러)      댄스   
233              언제나 사랑해   케이시 (Kassy)     발라드   

                                                                                                                                                                                                                                                                                                                                                                                                                                                         lyrics  \
357                                                                                      이젠 혼자가 아닐 무대 너무나 감격스러워 끝없는 가능성 중에 날 골라줘서 고마워 나와 맞이하는 미래

In [ ]:
# 일기 벡터 계산
diary_text = "첫눈 오는 이런 오후에 그에게 전화를 걸 수만 있다면 기쁠텐데 1년 전으로 갈 수 있다면 지금 우린 달라졌을까? 눈물 차 올라서 아무말도 못하고, 후회만 가득가득 했던 그 크리스마스가 생각나"
diary_vector = vectorizer.get_first_hidden_state(diary_text)

In [ ]:
# 코사인 유사도 계산 및 상위 3개의 도서 선택
similarities4 = []
idx = 0
for book_vector in book_vectors4:
    similarity4 = cosine_similarity(diary_vector.reshape(1, -1), book_vector.reshape(1, -1))
    similarities4.append((similarity4[0][0], idx))
    idx += 1

# 유사도가 높은 상위 3개의 음악 선택
top_indices4 = sorted(similarities4, key=lambda x : (x[0], x[1]))
top_indices4 = [top_indices4[i][1] for i in range(1, 6)]
top_books4 = books_df4.iloc[top_indices4]

for index in top_indices4:
  print(similarities4[index])

# 결과 출력
print("유사도가 높은 상위 3개의 음악:")
print(top_books4[['song_title','artist','genre', 'lyrics','emotion','tag','plylstseq']])

(0.05041421, 294)
(0.05074779, 408)
(0.08788961, 428)
(0.091817655, 256)
(0.09397701, 562)
유사도가 높은 상위 3개의 음악:
           song_title       artist genre  \
294             Siren        RIIZE    댄스   
408         New Jeans     NewJeans    댄스   
428       Baggy Jeans        NCT U    댄스   
256               미스터           카라    댄스   
562  Cheeky Icy Thang  STAYC(스테이씨)    댄스   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [ ]:
import pandas as pd
import requests
from IPython.display import Image, display
from PIL import Image as PILImage
from io import BytesIO
# 앨범 커버 이미지를 표시하는 함수 정의
def show_album_covers(df, column='album_cover'):
    for url in df[column].head(5):  # 상위 5개의 이미지만 표시
        response = requests.get(url)
        img = PILImage.open(BytesIO(response.content))
        display(img)

# 앨범 커버 이미지 시각화
show_album_covers(top_books6)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import pandas as pd
import requests
from PIL import Image as PILImage
from io import BytesIO
import base64
import json


# 이미지 URL을 base64 인코딩하여 JSON으로 저장하는 함수 정의
def save_images_to_json(df, column='album_cover', json_file='album_covers.json'):
    images_dict = {}
    for url in df[column].head(5):  # 상위 5개의 이미지만 처리
        response = requests.get(url)
        img = PILImage.open(BytesIO(response.content))
        buffered = BytesIO()
        img.save(buffered, format="JPEG")
        img_str = base64.b64encode(buffered.getvalue()).decode()
        images_dict[url] = img_str

    with open(json_file, 'w') as f:
        json.dump(images_dict, f)

# 이미지 URL을 base64 인코딩하여 JSON으로 저장
# 이미지 URL을 base64 인코딩하여 JSON으로 저장
json_file = 'album_covers.json'
save_images_to_json(top_books6, json_file=json_file)

# 파일 저장 확인
print(f"이미지가 '{json_file}' 파일로 저장되었습니다.")

이미지가 'album_covers.json' 파일로 저장되었습니다.


In [ ]:
import base64
import json
from PIL import Image as PILImage
from io import BytesIO
import os

# JSON 파일에서 이미지를 불러와 디코딩 및 파일로 저장하는 함수 정의
def load_images_from_json(json_file='album_covers.json', output_folder='output_images'):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    with open(json_file, 'r') as f:
        images_dict = json.load(f)

    for idx, (url, img_str) in enumerate(images_dict.items(), start=1):
        img_data = base64.b64decode(img_str)
        img = PILImage.open(BytesIO(img_data))

        # 이미지 파일 이름 생성
        img_name = f"image_{idx}.jpg"  # 고유한 파일 이름 생성

        img.save(f"{output_folder}/{img_name}")
        print(f"이미지가 '{output_folder}/{img_name}' 파일로 저장되었습니다.")

# JSON 파일에서 이미지를 불러와 디코딩 및 파일로 저장
load_images_from_json(json_file='album_covers.json', output_folder='output_images')


이미지가 'output_images/image_1.jpg' 파일로 저장되었습니다.
이미지가 'output_images/image_2.jpg' 파일로 저장되었습니다.
이미지가 'output_images/image_3.jpg' 파일로 저장되었습니다.
이미지가 'output_images/image_4.jpg' 파일로 저장되었습니다.
이미지가 'output_images/image_5.jpg' 파일로 저장되었습니다.


In [ ]:
import pandas as pd
import requests
from PIL import Image as PILImage
from io import BytesIO
import base64

# 예제 데이터프레임 생성 (이미 존재하는 top_books6 데이터프레임을 사용)
# 데이터 로드
books_df4 = pd.read_csv('/content/songs_with_tags2.csv', sep='\t')  # 가사 데이터 로드

# 이미지 URL을 base64 인코딩하여 데이터 프레임에 추가하는 함수 정의
def add_images_to_dataframe(df, column='album_cover'):
    base64_images = []
    for url in df[column]:  # 상위 5개의 이미지만 처리
        try:
            response = requests.get(url, stream=True)
            response.raise_for_status()  # HTTP 오류 발생 시 예외 발생
            img = PILImage.open(BytesIO(response.content))
            buffered = BytesIO()
            img.save(buffered, format="JPEG")
            img_str = base64.b64encode(buffered.getvalue()).decode()
            base64_images.append(img_str)
        except Exception as e:
            print(f"Error processing {url}: {e}")
            base64_images.append(None)  # 유효하지 않은 이미지는 None으로 처리

    df['album_cover_base64'] = pd.Series(base64_images)

# 이미지 URL을 base64 인코딩하여 데이터 프레임에 추가
add_images_to_dataframe(books_df4)

# 데이터 프레임을 CSV 파일로 저장
csv_file = 'songs_tag2_with_images.csv'
books_df4.to_csv(csv_file, index=False)

# 파일 저장 확인
print(f"데이터 프레임이 '{csv_file}' 파일로 저장되었습니다.")


데이터 프레임이 'songs_tag2_with_images.csv' 파일로 저장되었습니다.


In [ ]:
# 데이터 프레임을 CSV 파일로 저장
csv_file = 'songs_tag2_with_images.csv'
books_df4.to_csv(csv_file, index=False)


In [ ]:
import requests
import base64
import json
import pandas as pd

# 데이터셋 로드 (예시로 CSV 파일 사용)
df = pd.read_csv('dataset.csv')

# 이미지 URL을 Base64 형식으로 인코딩하여 JSON 열 추가
def url_to_base64_json(image_url):
    try:
        response = requests.get(image_url)
        response.raise_for_status()  # 요청이 성공했는지 확인
        encoded_string = base64.b64encode(response.content).decode('utf-8')
        return json.dumps({'image': encoded_string})
    except requests.RequestException as e:
        print(f"Error fetching image from {image_url}: {e}")
        return json.dumps({'image': None})

# 이미지 URL 열을 순회하며 JSON 열 생성
df['image_json'] = df['image_url'].apply(url_to_base64_json)

# 결과를 CSV 파일로 저장 (선택사항)
df.to_csv('updated_dataset.csv', index=False)

# 결과 출력 (선택사항)
print(df)
